this notebook is used to clean the raw data of ML-32M database and save it in usable format

In [2]:
import pandas as pd
import re

In [3]:
movies_df = pd.read_csv("data/movies.csv")
ratings_df = pd.read_csv("data/ratings.csv")
links_df = pd.read_csv("data/links.csv")
tags_df = pd.read_csv("data/tags.csv")

In [4]:
# --- Data Merging and Cleaning ---

# Calculate average rating and rating count
movie_ratings = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
movie_ratings.rename(columns={'mean': 'average_rating', 'count': 'rating_count'}, inplace=True)

# Merge movies with ratings
df = pd.merge(movies_df, movie_ratings, on='movieId', how='left')

# Merge with links to get tmdbId
df = pd.merge(df, links_df, on='movieId', how='left')

In [5]:
# --- Data Merging and Cleaning ---

# Calculate average rating and rating count
movie_ratings = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
movie_ratings.rename(columns={'mean': 'average_rating', 'count': 'rating_count'}, inplace=True)

# Merge movies with ratings
df = pd.merge(movies_df, movie_ratings, on='movieId', how='left')

# Merge with links to get tmdbId
df = pd.merge(df, links_df, on='movieId', how='left')

In [6]:
# # --- Process and Add Tags ---
# # Group all tags for each movie into a single pipe-separated string.
# tags_df['tag'] = tags_df['tag'].astype(str).str.lower()
# movie_tags = tags_df.groupby('movieId')['tag'].apply(lambda x: ' | '.join(x)).reset_index()
# movie_tags.rename(columns={'tag': 'tags'}, inplace=True) # Rename column to 'tags'

# # Merge the combined tags into the main dataframe
# df = pd.merge(df, movie_tags, on='movieId', how='left')
# # Fill movies that have no tags with an empty string
# df['tags'].fillna("", inplace=True)

In [7]:
# --- Feature Engineering ---

# 1. Separate title and release year
df['year'] = df['title'].str.extract(r'\((\d{4})\)', expand=False).fillna(0).astype(int)
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True).str.strip()

# 2. Round average rating
df['average_rating'] = df['average_rating'].fillna(0).round(3)
df['rating_count'] = df['rating_count'].fillna(0)

# 3. One-hot encode genres
genres_dummies = df['genres'].str.get_dummies(sep='|')

# Join the new genre columns to the main dataframe
df = pd.concat([df, genres_dummies], axis=1)

In [8]:
# --- Final DataFrame Preparation ---

# Drop original columns that are no longer needed
df.drop(['movieId', 'genres', 'imdbId'], axis=1, inplace=True)

# Handle cases where a tmdbId might be missing and set type
df.dropna(subset=['tmdbId'], inplace=True)
df['tmdbId'] = df['tmdbId'].astype(int)

# Reorder columns for better readability
genre_cols = genres_dummies.columns.tolist()
# New column order: identifiers, features, genres, and finally the new tags column
new_order = ['tmdbId', 'title', 'year', 'average_rating', 'rating_count'] + genre_cols #+ ['tags']
final_df = df[new_order]

# Display the first few rows of the transformed data
print("Transformed Data Head:")
print(final_df.head())
print("\nShape of the final DataFrame:", final_df.shape)

Transformed Data Head:
   tmdbId                        title  year  average_rating  rating_count  \
0     862                    Toy Story  1995           3.897       68997.0   
1    8844                      Jumanji  1995           3.276       28904.0   
2   15602             Grumpier Old Men  1995           3.139       13134.0   
3   31357            Waiting to Exhale  1995           2.845        2806.0   
4   11862  Father of the Bride Part II  1995           3.060       13154.0   

   (no genres listed)  Action  Adventure  Animation  Children  ...  Film-Noir  \
0                   0       0          1          1         1  ...          0   
1                   0       0          1          0         1  ...          0   
2                   0       0          0          0         0  ...          0   
3                   0       0          0          0         0  ...          0   
4                   0       0          0          0         0  ...          0   

   Horror  IMAX  Musi

In [9]:
# Save the result to a new CSV file
final_df.to_csv("movie_features.csv", index=False)